# Tour with TensorFlow and Keras

## Install dependencies

In [ ]:
! pip install --quiet neptune-client==0.4.131 neptune-contrib==0.25.0 tensorflow==2.3.1

# Basic Tour

* This tour will introduce you to Neptune client API. In the next few steps you will see how to create experiment in the project and log meaningful training data to the running experiment.
* This tour use popular deep learning libraries TensorFlow and Keras to make it looks familiar.

## Step 1: Import Neptune and TensorFlow

In [ ]:
import neptune
import tensorflow as tf

## Step 2: Select Neptune project

Use ``neptune.init`` to set project. It's necessary to tell Neptune to which project you want to log experiments' data.

**Note**: consider using your own project and API token. Here is how to:
* [get api token](https://docs.neptune.ai/security-and-privacy/api-tokens/how-to-find-and-set-neptune-api-token.html)
* [create project](https://docs.neptune.ai/workspace-project-and-user-management/projects/create-project.html)

In [ ]:
neptune.init('shared/tour-with-tf-keras',
             api_token='ANONYMOUS')

## Step 3: Create Neptune experiment

* Use ``neptune.create_experiment`` to make new experiment in the project.

In [ ]:
neptune.create_experiment(name='tf-keras-training-basic')

## Step 4: Prepare dataset and model

Use Keras API to prepare dataset (fashion MNIST) and model (feedforward network).

In [ ]:
# dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Step 5: Use NeptuneMonitor callback to log metrics during training

* Neptune has TF / Keras integration available. Simply use ``NeptuneMonitor`` callback in the ``fit`` method to log metrics during training.
* In the Neptune UI you will see metrics being updated almost in real time.

In [ ]:
from neptunecontrib.monitoring.keras import NeptuneMonitor

model.fit(x_train, y_train,
          epochs=10,
          validation_split=0.2,
          callbacks=[NeptuneMonitor()])

## Step 6: Log model evaluation metrics

Use ``neptune.log_metric`` to log model evaluation metrics.

In [ ]:
eval_metrics = model.evaluate(x_test, y_test, verbose=0)

for j, metric in enumerate(eval_metrics):
    neptune.log_metric('test_{}'.format(model.metrics_names[j]), metric)

## Step 7: Stop experiment at the end

This statement is needed only in Notebooks. In scripts experiment is closed automatically at the end.

In [ ]:
neptune.stop()

# More logging options

Below we present more logging options. Flow of the example is similar to the basic one. The difference is that here we present more logging options.

* logging experiments parameters,
* logging data version,
* model architecture summary,
* weights of the trained model,
* predictions on the test set as table,
* visualizations of the model performance,
* training data sample.

## Install additional dependencies

In [ ]:
! pip install --quiet scikit-plot==0.3.7 matplotlib==3.3.3

## Select Neptune project

Use ``neptune.init`` to set project. It's necessary to tell Neptune to which project you want to log experiments' data.

In [ ]:
neptune.init('shared/tour-with-tf-keras',
             api_token='ANONYMOUS')

## Prepare params

Prepare Python dict with parameters. You will log them to experiment at it's creation.

In [ ]:
parameters = {'dense_units': 32,
              'activation': 'relu',
              'dropout': 0.3,
              'learning_rate': 0.05,
              'batch_size': 32,
              'n_epochs': 10}

## Create Neptune experiment and log parameters

* Use ``neptune.create_experiment`` to make new experiment in the project.
* Log parameters when you create experiment.
* Add tags to experiment to better organize it in the project.

In [ ]:
neptune.create_experiment(name='tf-keras-training-advanced',
                          tags=['keras', 'fashion-mnist'],
                          params=parameters)

## Prepare dataset and log data version

* Use Keras API to prepare dataset (fashion MNIST).
* Use `set_property` method to log data version to the experiment.

In [ ]:
import hashlib

# prepare dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# log data version
neptune.set_property('x_train_version', hashlib.md5(x_train).hexdigest())
neptune.set_property('y_train_version', hashlib.md5(y_train).hexdigest())
neptune.set_property('x_test_version', hashlib.md5(x_test).hexdigest())
neptune.set_property('y_test_version', hashlib.md5(y_test).hexdigest())

neptune.set_property('class_names', class_names)

## Prepare model and log model architecture summary

* Use Keras API to prepare the model (feedforward network).
* Use `log_text` method to log model archoitecture summary.

In [ ]:
# prepare model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(parameters['dense_units'], activation=parameters['activation']),
    tf.keras.layers.Dropout(parameters['dropout']),
    tf.keras.layers.Dense(parameters['dense_units'], activation=parameters['activation']),
    tf.keras.layers.Dropout(parameters['dropout']),
    tf.keras.layers.Dense(10, activation='softmax')
])
optimizer = tf.keras.optimizers.SGD(learning_rate=parameters['learning_rate'])
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# log model summary
model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))

## Use NeptuneMonitor callback to log metrics during training

* Neptune has TF / Keras integration available. Simply use ``NeptuneMonitor`` callback in the ``fit`` method to log metrics during training.
* In the Neptune UI you will see metrics being updated almost in real time.

In [ ]:
model.fit(x_train, y_train,
          batch_size=parameters['batch_size'],
          epochs=parameters['n_epochs'],
          validation_split=0.2,
          callbacks=[NeptuneMonitor()])

## Log model evaluation metrics

Use ``neptune.log_metric`` to log model evaluation metrics.

In [ ]:
eval_metrics = model.evaluate(x_test, y_test, verbose=0)

for j, metric in enumerate(eval_metrics):
    neptune.log_metric('test_{}'.format(model.metrics_names[j]), metric)

## Log model weights after training

Use ``log_artifact`` method to log model weight files.

In [ ]:
model.save('model')
neptune.log_artifact('model')

## Log predictions as table

Use ``log_table`` method to log predictions dataframe as table.

In [ ]:
import numpy as np
import pandas as pd
from neptunecontrib.api import log_table

y_pred_proba = model.predict(x_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred = y_pred
df = pd.DataFrame(data={'y_test': y_test, 'y_pred': y_pred, 'y_pred_probability': y_pred_proba.max(axis=1)})
log_table('predictions', df)

## Log model performance visualizations

Use ``log_image`` to log ROC and precision recall charts.

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_roc, plot_precision_recall

fig, ax = plt.subplots()
plot_roc(y_test, y_pred_proba, ax=ax)
neptune.log_image('model-performance-visualizations', fig, image_name='ROC')

fig, ax = plt.subplots()
plot_precision_recall(y_test, y_pred_proba, ax=ax)
neptune.log_image('model-performance-visualizations', fig, image_name='precision recall')
plt.close('all')

## Log train data sample (images per class)

Use ``log_image`` to log sample of 9 images for each class in the training dataset.

In [ ]:
for j, class_name in enumerate(class_names):
    plt.figure(figsize=(10, 10))
    label_ = np.where(y_train == j)
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(x_train[label_[0][i]], cmap=plt.cm.binary)
        plt.xlabel(class_names[j])
    neptune.log_image('train data sample', plt.gcf())
    plt.close('all')

## Stop experiment at the end

This statement is needed only in Notebooks. In scripts experiment is closed automatically at the end.

In [ ]:
neptune.stop()